In [1]:
# import modules
import requests
import json

In [3]:
# football data from https://www.api-football.com/
baseURL = "https://v3.football.api-sports.io"
payload={}
headers = {
  'x-rapidapi-key': 'xxx', # use api key
  'x-rapidapi-host': 'v3.football.api-sports.io'
}

In [ ]:
# 1. fetch all leagues (Result: see "./mock-data/raw_leagues.json")
response = requests.request("GET", baseURL + "/leagues", headers=headers, data=payload)
jsonData = response.json()
with open("../data/raw_leagues.json", "w") as write_file:
    json.dump(jsonData['response'], write_file)

In [ ]:
# 2. based on all leagues, fetch single seasons where player stats is covered (Result: see "./mock-data/raw_leagues.json")

# open raw_leagues.json data
with open('../data/raw_leagues.json') as file:
    leagues = json.load(file)

seasons_data = [] # init empty list
uniqueIdCount = 1 # add artificial seasons id

for league in leagues:
    for season in league['seasons']:
        if season['coverage']['players'] == True: # save only seasons with player coverage
            seasons_data.append({
                'seasonID': uniqueIdCount,
                'leagueID': league['league']['id'],
                'name': league['league']['name'], 
                'type': league['league']['type'], 
                'logo': league['league']['logo'],
                'country': league['country']['name'],
                'year': season['year'],
                'start': season['start'],
                'end': season['end']
            })
            uniqueIdCount += 1

# save seasons data as json
with open('../data/raw_seasons.json', 'w', encoding='utf-8') as f:
    json.dump(seasons_data, f, ensure_ascii=False, indent=4)

In [ ]:
# 3. fetch all players based on seasons (Result: see "./mock-data/raw_players.json")

# open seasons json data
with open('../data/raw_seasons.json') as file:
    seasons = json.load(file)
    
players_data = [] # init empty list

# iterate over all seasons
for season in seasons:
    url = baseURL + "/players?league={leagueID}&season={seasonYear}".format(leagueID = season['leagueID'], seasonYear = season['year'])
    response = requests.request("GET", url + "", headers=headers, data=payload)
    jsonData = response.json()
    current = jsonData['paging']['current'] # for pagination
    total = jsonData['paging']['total'] # for pagination
    
    # pagination
    for page in range(current, total + 1):
        url = baseURL + "/players?league={leagueID}&season={seasonYear}&page={page}".format(leagueID = season['leagueID'], seasonYear = season['year'], page = page)
        response = requests.request("GET", url, headers=headers, data=payload)
        jsonData = response.json()
        results = jsonData['response']
        for player in results:
            # add player to list
            players_data.append({
                'leagueID': season['leagueID'],
                'seasonID': season['seasonID'],
                'seasonYear': season['year'],
                'player': player['player'],
                'statistics': player['statistics']
            })

# create raw_players.json file
with open('../data/raw_players.json', 'w', encoding='utf-8') as f:
    json.dump(players_data, f, ensure_ascii=False, indent=4)
